In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb

In [ ]:
covid_df = pd.read_csv('../input/covid19-sentiments/COVID-19_Sentiments.csv')
covid_df.head()

In [ ]:
covid_df.shape

In [ ]:
if covid_df.duplicated().any():
    print('Duplicates Found')
else:    
    print('No Duplicates Found')

In [ ]:
covid_df = covid_df.drop_duplicates().reset_index(drop=True)

In [ ]:
covid_df.shape

In [ ]:
covid_df.isna().sum()

In [ ]:
covid_df.Sentiments.fillna(covid_df['Sentiments'].mean(),inplace=True)

In [ ]:
covid_df.isna().sum()

In [ ]:
covid_df.head()

In [ ]:
covid_df.Sentiments.value_counts()

In [ ]:
def response(x):
    if x > 0:
        return 1
    elif x < 0:
        return -1
    else :
        return 0
covid_df['Sentiments'] = covid_df['Sentiments'].apply(response)


In [ ]:
covid_df.Sentiments.value_counts()

In [ ]:
covid_df.head(10)

In [ ]:
covid_df.drop(['Text_Id', 'Date', 'Location'], axis =1, inplace = True)


In [ ]:
covid_df = covid_df[covid_df.Sentiments != 0]

In [ ]:
covid_df.head()

Data Cleaning

In [ ]:
import string
string.punctuation

def remove_punctuation(text):
    no_punct = [words for words in text if words not in string.punctuation]
    words_wo_punct = ''.join(no_punct)
    return words_wo_punct
covid_df["Text_wo_punct"] = covid_df['Text'].apply(lambda x: remove_punctuation(x))
covid_df.head()
    

In [ ]:
from nltk.tokenize import RegexpTokenizer
import re
tokenizer = RegexpTokenizer(r'\w+') 
covid_df['Text_wo_punct'] = covid_df['Text_wo_punct'].apply(lambda x: tokenizer.tokenize(x.lower()))
covid_df.head()

In [ ]:
from nltk.corpus import stopwords
stop = list(stopwords.words('english'))
remove = ['i','a','d','o','y']
stop = [ele for ele in stop if ele not in remove ]
print(len(stop))

def remove_stopwords(text):
    words = [words for words in text if words not in stop]
    return words
covid_df['Text_wo_punct_wo_sw'] = covid_df['Text_wo_punct'].apply(lambda x: remove_stopwords(x))
covid_df.head()

In [ ]:
from nltk.stem import WordNetLemmatizer
lemm = WordNetLemmatizer()
def lemmatise(text):
    lem_text = ' '.join([lemm.lemmatize(words) for words in text])
    return lem_text
covid_df['Final_Text'] = covid_df['Text_wo_punct_wo_sw'].apply(lambda x: lemmatise(x))
covid_df

**Training Model for prediction **

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(covid_df.Final_Text, covid_df.Sentiments, test_size = 0.3, random_state = 1)

In [ ]:
good = x_train[y_train[y_train == 1].index]
#neutral = x_train[y_train[y_train == 0].index]
bad = x_train[y_train[y_train == -1].index]

In [ ]:
from wordcloud import WordCloud


plt.figure(figsize = (20,20))
wc = WordCloud(min_font_size = 3,  max_words = 3000 , width = 1600 , height = 800).generate(" ".join(good))

plt.imshow(wc,interpolation = 'bilinear')

In [ ]:
#plt.figure(figsize = (20,20))
#wc = WordCloud(min_font_size = 3, max_words = 3000, width =1600, height = 800).generate(" ".join(neutral))
#plt.imshow(wc,interpolation = 'bilinear')

In [ ]:
plt.figure(figsize = (20,20))
wc = WordCloud(min_font_size = 3, max_words = 3000, width = 1600, height = 800).generate(" ".join(bad))
plt.imshow(wc, interpolation = 'bilinear')

In [ ]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
obj = TfidfVectorizer(min_df=0,max_df=1,use_idf=True,ngram_range=(1,3))
obj_xtrain = obj.fit_transform(x_train)
obj_xtest = obj.transform(x_test)
print(obj_xtrain.shape)
print(obj_xtest.shape)


In [ ]:
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.linear_model import LogisticRegression
lr=LogisticRegression(penalty='l2',max_iter=500,C=1,random_state=0)
lr_tfidf=lr.fit(obj_xtrain,y_train)

In [ ]:
#lr_tfidf_predict = lr.predict(obj_xtest)
print(lr.score(obj_xtest,y_test))

In [ ]:
#lr_tfidf_score=accuracy_score(y_test,lr_tfidf_predict)
#print(lr_tfidf_score)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB()
classifier.fit(obj_xtrain, y_train)

In [ ]:
#mnb_tfidf_predict = classifier.predict(obj_xtest)
print(classifier.score(obj_xtest, y_test))


In [ ]:
#mnb_tfidf_score = accuracy_score(y_test,mnb_tfidf_predict)
#print(mnb_tfidf_score)

In [ ]:
from sklearn.svm import LinearSVC

linear_svc = LinearSVC(C=0.5, random_state=42)
linear_svc.fit(obj_xtrain, y_train)

print(linear_svc.score(obj_xtest,y_test))